# 模型推理 - 使用 QLoRA 微调后的 ChatGLM3-6B

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

# 定义全局变量和参数
model_name_or_path = '/root/autodl-tmp/models/chatglm3-6b'  # 模型ID或本地路径
peft_model_path = '/root/autodl-tmp/models/chatglm-6b-qlora'


In [3]:
config = PeftConfig.from_pretrained(peft_model_path)

q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=torch.float32)

base_model = AutoModel.from_pretrained(config.base_model_name_or_path,
                                       quantization_config=q_config,
                                       trust_remote_code=True,
                                       device_map='auto')
base_model.requires_grad_(False)
base_model.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

## 微调前后效果对比


### ChatGLM3-6B

In [4]:
input_text = '坤卦是什么'
print(f'输入：\n{input_text}')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)

输入：
坤卦是什么


In [8]:
response, history = base_model.chat(tokenizer=tokenizer, query=input_text)
print(f'ChatGLM3-6B 微调前：\n{response}')

ChatGLM3-6B 微调前：
坤卦是《易经》中的一个卦象,由两个阴爻夹一个阳爻构成。这个卦象象征着“地”,也代表着柔顺、承载、包容、承载、承载、承载、承载等含义。

坤卦是八卦中属于“地”卦,是“坤”卦,具有包容、承载、承载、承载、承载等含义。在《易经》中,坤卦代表地,也象征着柔顺、承载、包容、承载、承载等含义。

坤卦是八卦中属于“地”卦,具有包容、承载、承载、承载等含义。在《易经》中,坤卦代表地,也象征着柔顺、承载、包容、承载、承载等含义。


In [7]:
model = PeftModel.from_pretrained(base_model, peft_model_path)
response, history = model.chat(tokenizer=tokenizer, query=input_text)
print(f'ChatGLM3-6B 微调后: \n{response}')

ChatGLM3-6B 微调后: 
坤卦是八卦之一，由两个阴爻夹一个阳爻构成，象征着地，也代表着顺从、承载、滋养、承载等含义。在八卦中，坤卦代表地地重坤，具有顺从、承载、滋养、承载等含义。在六十四卦中，坤卦是柔顺、承载、滋养的象征，具有顺从、承载、滋养、承载等含义。
